In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

Using TensorFlow backend.


In [2]:
model = load_model('emojinator.h5')

In [3]:
def main():
    
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x,y,w,h = 300,50,350,350
    
    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, np.array([2,50,60]), np.array([25,150,255]))
        res = cv2.bitwise_and(img, img, mask=mask)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5,5), 0)
        
        kernel_square = np.ones((5,5), dtype = np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30,255, cv2.THRESH_BINARY)
        
        thresh = thresh[y:y+h, x:x+w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        
        if len(contours) >0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x,y,w1,h1 = cv2.boundingRect(contour)
                newImg = thresh[y:y+h1, x:x+w1]
                newImg = cv2.resize(newImg, (50,50))
                pred_prob, pred_class = predict(model, newImg)
                print(pred_prob, pred_class)
                img = overlay(img, emojis[pred_class], 400, 250,90,90)
                
        x,y,w,h = 300,50,350,350
        cv2.imshow('frame', img)
        cv2.imshow('Contours', thresh)
        k = cv2.waitKey(10)
        if k==27:
            break
    cap.release()
    cv2.destroyAllWindows()


In [4]:
def predict(model, img):
    processed = process_img(img)
    pred_prob = model.predict(processed)[0]
    pred_class = list(pred_prob).index(max(pred_prob))
    return max(pred_prob), pred_class

In [5]:
def process_img(img):
    image_x=50 
    image_y=50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [6]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis=[]
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

In [7]:
def overlay(image, emoji, x,y,w,h):
#     cv2.imshow('sds', emoji)
    emoji = cv2.resize(emoji, (w,h))
    cv2.imshow('sds', emoji)

    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h,x:x+w], emoji)
        cv2.imshow('sds', image)

    except:
        pass
    return image

In [8]:
def blend_transparent(face_img, overlay_t_img):
    overlay_img = overlay_t_img[:,:,:3]
    overlay_mask = overlay_t_img[:,:,3:]
    
    background_mask = 255-overlay_mask
    
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLORGRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)
    
    face_part = (face_img * (1/255.0)) * (background_mask * (1/255.0))
    overlay_part = (overlay_img * (1/255.0)) * (overlay_mask * (1/255.0))
    
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [9]:
predict(model, np.zeros((50,50,1), dtype=np.uint8))
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
1.0 11
1.0 11
1.0 5
1.0 8
1.0 5
1.0 8
1.0 11
0.9995096 8
1.0 11
1.0 11
1.0 11
1.0 11
1.0 11
1.0 8
1.0 11
1.0 5
1.0 11
1.0 11
1.0 8
1.0 8
1.0 11
1.0 8
1.0 5
1.0 11
1.0 11
1.0 5
1.0 11
0.9999988 5
1.0 5
1.0 11
1.0 8
1.0 11
1.0 11
1.0 5
1.0 5
1.0 11
1.0 11
1.0 5
1.0 8
1.0 11
1.0 11
1.0 11
0.9940587 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 11
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 8
1.0 11
1.0 8
1.0 11
1.0 5
1.0 11
1.0 8
1.0 11
1.0 11
1.0 2
1.0 5
1.0 5
1.0 11
1.0 11
1.0 11
1.0 11
1.0 11
1.0 8
1.0 5
1.0 11
1.0 8
1.0 11
1.0 11
1.0 8
1.0 11
1.0 11
1.0 11
1.0 11
1.0 11
1.0 11
1.0 5
1.0 8
1.0 11
1.0 5
1.0 2
1.0 11
1.0 11
1.0 11
1.0 11
1.0 8
1.0 5
1.0 11
1.0 5
1.0 11
1.0 11
1.0 11
1.0 11
1.0 11
1.0 5
1.0 11
1.0 5
1.0 5
1.0 11
1.0 11
1.0 11
1.0 8
1.0 5
1.0 4
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1.0 2
1.0 5
1.0 5
1.0 5
1.0 5
1.0 5
1